In [1]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

# Commonly used modules
import numpy as np
import os
import sys

# Images, plots, display, and visualization
import matplotlib.pyplot as plt
import pandas as pd

import seaborn as sns
import cv2
import IPython
from six.moves import urllib

print(tf.__version__)

2023-06-27 11:57:18.629169: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-27 11:57:18.680341: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-27 11:57:19.454418: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


2.12.0


In [2]:
# # Code for loading Data
# dataset = keras.preprocessing.image_dataset_from_directory(
#     directory="/var/trainingData/",
#     labels="inferred",
#     label_mode="int",
#     batch_size=32,
#     image_size=(180, 180),
#     shuffle=True,
#     seed=123,
# )

In [3]:
# Assume your directory structure is:
# main_directory/
# ...class_a/
# ......a_image_1.jpg
# ......a_image_2.jpg
# ...class_b/
# ......b_image_1.jpg
# ......b_image_2.jpg

# Create a training dataset from the main directory with 70% of data
train_ds = tf.keras.utils.image_dataset_from_directory(
  directory="/var/trainingData/",
  labels="inferred",
  label_mode="int",
  image_size=(180, 180),
  batch_size=32,
  shuffle=True,
  seed=123,
  validation_split=0.003,
  subset="validation",
)

# Create a validation dataset from the main directory with 30% of data
val_ds = tf.keras.utils.image_dataset_from_directory(
  directory="/var/trainingData/",
  labels="inferred",
  label_mode="int",
  image_size=(180, 180),
  batch_size=32,
  shuffle=True,
  seed=123,
  validation_split=0.0003,
  subset="validation",
)

Found 1177313 files belonging to 2002 classes.
Using 3531 files for validation.


2023-06-27 11:58:00.697505: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-27 11:58:00.701230: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-27 11:58:00.704675: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Found 1177313 files belonging to 2002 classes.
Using 353 files for validation.


In [4]:
# # Get the class names from the train_ds
# class_names = train_ds.class_names

# # Print the class names
# print(class_names)

# # Get the number of labels
# num_labels = len(class_names)

# # Print the number of labels
# print(num_labels)


In [5]:
# # Define a filter function that checks if the image has a non-zero shape
# def filter_zero_images(image, label):
#   # Get the shape of the image
#   image_shape = tf.shape(image)
#   # Check if the shape is non-zero
#   non_zero = tf.math.reduce_any(image_shape > 0)
#   # Return True or False
#   return non_zero

# # Apply the filter function to your batch dataset
# train_ds = train_ds.filter(filter_zero_images)

In [6]:
# # Get the number of elements in the dataset as a tensor
# num_elements = tf.data.experimental.cardinality(train_ds)

# # Convert the tensor to a Python integer
# num_elements = tf.get_static_value(num_elements)

# # Print the number of elements
# print(num_elements)


In [7]:
# # Get the batch size
# batch_size = 32

# # Get the number of images by multiplying the number of elements by the batch size
# num_images = num_elements * batch_size

# # Print the number of images
# print(num_images)


In [8]:
# import torchvision.transforms as transforms

# # Define a transform to convert PIL images to tensors
# transform = transforms.ToTensor()

# # Iterate through each sample in the train_dataset
# for sample in train_dataset:
#     # Get the image tensor from the sample
#     image_tensor = sample[0]

#     # Check if the image tensor is empty
#     if image_tensor.numel() == 0:
#         print("Empty image found!")

#     # If you want to convert the image tensor to a PIL image for further processing:
#     # Convert the image tensor to a PIL image
#     image_pil = transforms.ToPILImage()(image_tensor)

#     # Check if the PIL image dimensions are zero
#     width, height = image_pil.size
#     if width == 0 or height == 0:
#         print("Empty image found!")

In [9]:
# Define a custom layer that wraps the resize_with_pad function
class ResizeWithPadLayer(tf.keras.layers.Layer):
  def __init__(self, target_height, target_width):
    super(ResizeWithPadLayer, self).__init__()
    self.target_height = target_height
    self.target_width = target_width

  def call(self, inputs):
    return tf.image.resize_with_pad(inputs, self.target_height, self.target_width)

# Create an instance of the custom layer
resize_with_pad_layer = ResizeWithPadLayer(180, 180)


In [10]:
# Import the necessary modules
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Define the model architecture
model = keras.Sequential([
  # Rescale the pixel values to the range [0, 1]
  layers.Rescaling(1./255, input_shape=(180, 180, 3)),
  # Apply a convolutional layer with 16 filters and a 3x3 kernel
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  # Apply a max pooling layer with a 2x2 window
  layers.MaxPooling2D(),
  # Apply another convolutional layer with 32 filters and a 3x3 kernel
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  # Apply another max pooling layer with a 2x2 window
  layers.MaxPooling2D(),
  # Apply a dropout layer with a rate of 0.2 to reduce overfitting
  layers.Dropout(0.2),
  # Flatten the output of the previous layer
  layers.Flatten(),
  # Apply a dense layer with 128 units and a ReLU activation
  layers.Dense(128, activation='relu'),
  # Apply another dropout layer with a rate of 0.2
  layers.Dropout(0.2),
  # Apply an output layer with the number of classes and a softmax activation
  layers.Dense(2002, activation='softmax')
])

# Compile the model with an optimizer, a loss function, and a metric
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Print the model summary
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 180, 180, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 180, 180, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 90, 90, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 90, 90, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 45, 45, 32)       0         
 2D)                                                             
                                                                 
 dropout (Dropout)           (None, 45, 45, 32)        0

In [11]:
# Train the model for 10 epochs
model.fit(train_ds, validation_data=val_ds, epochs=10)

# Evaluate the model on the test data
loss, accuracy = model.evaluate(val_ds)

# Print the loss and accuracy
print("Loss: ", loss)
print("Accuracy: ", accuracy)


Epoch 1/10


2023-06-27 11:58:42.991029: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [3531]
	 [[{{node Placeholder/_4}}]]
2023-06-27 11:58:42.991325: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [3531]
	 [[{{node Placeholder/_4}}]]
/usr/local/lib/python3.8/dist-packages/keras/backend.py:5612: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(
2023-06-27 11:58:43.

111/111 [==============================] - ETA: 0s - loss: 7.5991 - accuracy: 5.6641e-04

2023-06-27 11:58:49.081330: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [353]
	 [[{{node Placeholder/_4}}]]
2023-06-27 11:58:49.081626: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [353]
	 [[{{node Placeholder/_0}}]]


111/111 [==============================] - 6s 24ms/step - loss: 7.5991 - accuracy: 5.6641e-04 - val_loss: 7.4687 - val_accuracy: 0.0028
Epoch 2/10
111/111 [==============================] - 2s 21ms/step - loss: 7.2485 - accuracy: 5.6641e-04 - val_loss: 7.0351 - val_accuracy: 0.0057
Epoch 3/10
111/111 [==============================] - 2s 20ms/step - loss: 7.1210 - accuracy: 0.0014 - val_loss: 6.7395 - val_accuracy: 0.0142
Epoch 4/10
111/111 [==============================] - 2s 20ms/step - loss: 7.0189 - accuracy: 0.0042 - val_loss: 6.3629 - val_accuracy: 0.0142
Epoch 5/10
111/111 [==============================] - 2s 20ms/step - loss: 6.8590 - accuracy: 0.0062 - val_loss: 5.8838 - val_accuracy: 0.0312
Epoch 6/10
111/111 [==============================] - 2s 20ms/step - loss: 6.5805 - accuracy: 0.0127 - val_loss: 5.2861 - val_accuracy: 0.0850
Epoch 7/10
111/111 [==============================] - 2s 20ms/step - loss: 6.2666 - accuracy: 0.0255 - val_loss: 5.2969 - val_accuracy: 0.1048
Ep